In [1]:
# EuRoC data handler
import pandas as pd
import numpy as np

# Use the simple keras deep learning API
from sklearn.metrics import mean_squared_error
from keras import backend as K
from keras.models import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from math import sqrt
import time

# class euroc_data():
# 	def __init__(self,groundtruth_path):
# 		gt_frame = pd.read_csv(groundtruth_path,'r')
GROUNDTRUTH_PATH = '../data/state_groundtruth_estimate0/data.csv'
IMU_PATH = '../data/imu0/data.csv'

Using TensorFlow backend.


In [2]:
gt_dataframe = pd.read_csv(GROUNDTRUTH_PATH, index_col=0)
imu_dataframe = pd.read_csv(IMU_PATH, index_col=0)
# print(gt_dataframe.head(3))
# print('-----------------')
# print(imu_dataframe.head(3))

In [3]:
# Find the start and end time stamp of the ground truth, 
# and extract relevant imu data frame, gt and imu should be same length  
groudtruth_data = gt_dataframe.iloc[:-1,0:7]
start_time, end_time = groudtruth_data.index[0], groudtruth_data.index[-1]
print (start_time, end_time)
groudtruth_data_rela_pos = groudtruth_data.iloc[1:]-groudtruth_data.iloc[0,:]

1403636580838555648 1403636762738555392


In [4]:
groudtruth_data.head(5)

,p_RS_R_x [m],p_RS_R_y [m],p_RS_R_z [m],q_RS_w [],q_RS_x [],q_RS_y [],q_RS_z []
#timestamp,,,,,,,
1403636580838555648,4.688319,-1.786938,0.783338,0.534108,-0.153029,-0.827383,-0.082152
1403636580843555328,4.688177,-1.786770,0.787350,0.534640,-0.152990,-0.826976,-0.082863
1403636580848555520,4.688028,-1.786598,0.791382,0.535178,-0.152945,-0.826562,-0.083605
1403636580853555456,4.687878,-1.786421,0.795429,0.535715,-0.152884,-0.826146,-0.084391
1403636580858555648,4.687727,-1.786240,0.799484,0.536244,-0.152821,-0.825731,-0.085213


In [5]:
groudtruth_data_rela_pos.head(5)

,p_RS_R_x [m],p_RS_R_y [m],p_RS_R_z [m],q_RS_w [],q_RS_x [],q_RS_y [],q_RS_z []
#timestamp,,,,,,,
1403636580843555328,-0.000142,0.000168,0.004012,0.000532,0.000039,0.000407,-0.000711
1403636580848555520,-0.000291,0.000340,0.008044,0.001070,0.000084,0.000821,-0.001453
1403636580853555456,-0.000441,0.000517,0.012091,0.001607,0.000145,0.001237,-0.002239
1403636580858555648,-0.000592,0.000698,0.016146,0.002136,0.000208,0.001652,-0.003061
1403636580863555328,-0.000740,0.000879,0.020202,0.002660,0.000261,0.002069,-0.003897


In [6]:
# Set threshold to 2 000 000 ns, because the time interval between imu data is 5 000 000 ns
threshold = 1000000
[imu_start_idx, imu_end_idx] = [idx for (idx, timestamp) in enumerate(imu_dataframe.index)  \
    if (abs(timestamp-start_time)<threshold or abs(timestamp-end_time)<threshold)]
print (imu_start_idx, imu_end_idx)

216 36596


In [7]:
# ---------imu data at t-1 is used to predicate gt at t
# Groundtruth is for image 1-n, [first_image_middle_exp_time, last_image_middle_exp_time], that't why the number of 
# datapoints end with 1.
imu_data = imu_dataframe[imu_start_idx:imu_end_idx].copy()

if (groudtruth_data.shape[0] != imu_data.shape[0]):
    print("'Error: groundtruth and imu data don't have the same lenght'")
print (imu_data.shape)
print (groudtruth_data_rela_pos.shape)

imu_data.head(4)
# imu_data.iloc[1:3]
# imu_dataframe[0:6]

'Error: groundtruth and imu data don't have the same lenght'
(36380, 6)
(36380, 7)


,w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
#timestamp [ns],,,,,,
1403636580838555392,-0.206647,0.325329,-0.040492,10.035472,-0.179789,-4.135137
1403636580843555584,-0.215025,0.333009,-0.046775,9.749445,-0.024517,-4.077932
1403636580848555520,-0.224798,0.336499,-0.057945,9.496106,0.024517,-4.077932
1403636580853555456,-0.237365,0.338594,-0.074700,9.210079,0.032689,-4.061588


In [29]:

from keras import backend as K
# After studying the dataset, ignore the last data from ground truth. Because the second last element 
# has the timestamp that happens to be the middle point of image 3661 and 3662 (opened in Office, column index)
datasize = imu_data.shape[0] 
train_val_split_ratio = 0.7
training_size = int(datasize*train_val_split_ratio//10*10)
print(' Training size is: {} \n Testing size is: {}'.format(training_size, datasize-training_size))
training_input = imu_data[:training_size]
training_groudtruth = groudtruth_data_rela_pos[:training_size]
testing_input = imu_data[training_size:]
testing_groundtruth = groudtruth_data_rela_pos[training_size:]

training_input = np.array(training_input)
training_groudtruth = np.array(training_groudtruth)
testing_input = np.array(testing_input)
testing_groundtruth = np.array(testing_groundtruth)
# print (training_input.shape)
# print (training_groudtruth.shape)
# print (testing_input.shape)
# print (testing_groundtruth.shape)

# Find a way to record the error and some other data for visualizing
log_dir = '../tmp/logs/sensor_fusion_LSTM_rel_pos'


neurons = 64
batch_size = 1
num_epoch = 1000
#-------***************--------------#
# IMU 200Hz,   1 min = 1200 samples
series_length = 200
num_episodes  = (training_input.shape[0]//series_length)

train_X = training_input[:num_episodes*series_length,:].reshape(num_episodes, series_length, training_input.shape[1])
train_y = training_groudtruth[:num_episodes*series_length].reshape(num_episodes,series_length, training_groudtruth.shape[1])
#-------***************------------#
test_num_episodes = testing_input.shape[0]//series_length
test_X = testing_input[:test_num_episodes*series_length,:].reshape(test_num_episodes, series_length, testing_input.shape[1])
test_y = testing_groundtruth[:test_num_episodes*series_length].reshape(test_num_episodes, series_length, testing_groundtruth.shape[1])
# test_X = np.expand_dims(testing_input, axis=0)
# test_y = np.expand_dims(testing_groundtruth,axis=0)

train_X = train_X[:20]
train_y = train_y[:20]
test_X = test_X[:2]
test_y = test_y[:2]

print (train_X.shape)
print (train_y.shape)
print (test_X.shape)
print (test_y.shape)

def my_loss(y_true, y_pred):
    y_pred_normed = y_pred/K.repeat_elements(K.sqrt(K.sum(K.square(y_pred[:,:,3:]), axis=2, keepdims=True)), 7, axis=2)
    return K.mean(K.square(y_true-y_pred_normed), )

    
    
def my_metric(y_true, y_pred):
    y_pred_normed = y_pred/K.repeat_elements(K.sqrt(K.sum(K.square(y_pred[:,:,3:]), axis=2, keepdims=True)), 7, axis=2)
    return K.sqrt(K.square(y_true-y_pred_normed))

model = Sequential()
# input shape, when 3D (batch_size, timesteps, input_dim) input_dim=train_X.shape[2] input_shape=(None, train_X.shape[2]),batch_input_shape=(1,None,train_X.shape[2])
model.add(LSTM(neurons, input_shape=(200, train_X.shape[2]), stateful=True, batch_size=1, return_sequences=True))
model.add(TimeDistributed(Dense(64)))
model.add(TimeDistributed(Dense(7)))
# This optimizer is usually a good choice for recurrent neural networks.  optimizer='RMSprop'   metrics=[my_metric]
model.compile(loss=my_loss, metrics=[my_metric], optimizer='RMSprop')

tbCallBack = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, batch_size=batch_size)

hist = model.fit(train_X, train_y, epochs=num_epoch, batch_size=batch_size, verbose=1, validation_data=(test_X, test_y), shuffle=False)

model.reset_states()

# model = Sequential()
# model.add(LSTM(neurons, input_dim = train_X.shape[2], stateful=False, return_sequences=True))
# model.add(TimeDistributed(Dense(7)))
# model.compile(loss='mse', optimizer='adam')

# # plot_model(model, to_file='model.png')
# tbCallBack = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, batch_size=batch_size)
# # for i in range(num_epoch):
# #     start = time.time()
# hist = model.fit(train_X, train_y, epochs=num_epoch, batch_size=batch_size, verbose=1, 
#                      validation_data=(test_X, test_y), shuffle=False, callbacks=[tbCallBack] )

 Training size is: 25460 
 Testing size is: 10920
(20, 200, 6)
(20, 200, 7)
(2, 200, 6)
(2, 200, 7)
Train on 20 samples, validate on 2 samples
Epoch 1/1000
20/20 [==============================] - 19s - loss: 0.1596 - my_metric: 0.3033 - val_loss: 18.3085 - val_my_metric: 2.6664
Epoch 2/1000
 5/20 [======>.......................] - ETA: 11s - loss: 0.1608 - my_metric: 0.2964

KeyboardInterrupt: 

In [ ]:
a = tf.constant([1,2,3],dtype=np.float32 )
b = tf.constant([4,5,6])
with tf.Session() as sess:
    ans = tf.sqrt(tf.reduce_sum(tf.square(a)) )
    print(sess.run(ans))
    

In [ ]:
print (model.summary())
model.get_config()